In [1]:
%matplotlib inline
# %load_ext autoreload
# %autoreload 2

import sys; sys.path.insert(0, '../')
import pickle as pkl
import numpy as np
import pandas as pd
from copy import deepcopy
import mne
import seaborn as sns
import matplotlib.pyplot as plt
from esinet import util
from esinet import Simulation
from esinet import Net
from esinet import forward

plot_params = dict(surface='white', hemi='both', verbose=0)

# Forward Model

In [2]:
info = forward.get_info()
info['sfreq'] = 100
fwd = forward.create_forward_model(info=info)
fwd_free = forward.create_forward_model(info=info, fixed_ori=False)

[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   5 out of  16 | elapsed:    0.9s remaining:    2.1s
[Parallel(n_jobs=16)]: Done   9 out of  16 | elapsed:    0.9s remaining:    0.7s
[Parallel(n_jobs=16)]: Done  13 out of  16 | elapsed:    0.9s remaining:    0.1s
[Parallel(n_jobs=16)]: Done  16 out of  16 | elapsed:    0.9s finished
[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   5 out of  16 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=16)]: Done   9 out of  16 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=16)]: Done  13 out of  16 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=16)]: Done  16 out of  16 | elapsed:    0.0s finished
[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   5 out of  16 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=16)]: Done   9 out of  16 | elapsed:    0.0

# Load Models

In [3]:
lstm_standard = util.load_net('models/LSTM Medium_1-1000points_standard-cosine_0')

dense_standard = util.load_net('models/Dense Medium_1-1000points_standard-cosine_0')

convdip_standard = util.load_net('models/ConvDip Medium_1-1000points_standard-cosine_0')

models = [lstm_standard, dense_standard, convdip_standard]
model_names = ['LSTM', 'Fully-Connected', 'ConvDip']

# Plot single ground truth and predictions

In [5]:
import seaborn as sns
%matplotlib qt
sns.reset_orig()

%load_ext autoreload
%autoreload 2
model_names_tmp = deepcopy(model_names)
plot_params = dict(surface='white', hemi='both', verbose=0, 
    clim=dict(kind='percent', pos_lims=[20, 30, 100]))

settings_eval = dict(method='standard')

# Simulate new data
sim = Simulation(fwd, info, settings=settings_eval).simulate(2)
# snr = sim.simulation_info['target_snr'].values[0]
snr = None
# print(sim.simulation_info)
idx = 0
# Predict sources using the esinet models
predictions = [model.predict(sim) for model in models]

# Predict sources with classical methods
# eLORETA
prediction_elor_data = util.wrap_mne_inverse(fwd, sim, method='eLORETA', 
    add_baseline=True, n_baseline=400)[idx].data.astype(np.float32)
prediction_elor = deepcopy(predictions[0][0])
prediction_elor.data = prediction_elor_data / np.abs(np.max(prediction_elor_data))
# MNE
prediction_mne_data = util.wrap_mne_inverse(fwd, sim, method='MNE', 
    add_baseline=True, n_baseline=400)[idx].data.astype(np.float32)
prediction_mne = deepcopy(predictions[0][0])
prediction_mne.data = prediction_mne_data / np.abs(np.max(prediction_mne_data))
# Beamformer
prediction_lcmv_data = util.wrap_mne_inverse(fwd, sim, method='lcmv', 
    add_baseline=True, n_baseline=400, parallel=False)[idx].data.astype(np.float32)
prediction_lcmv = deepcopy(predictions[0][0])
prediction_lcmv.data = prediction_lcmv_data / np.max(np.abs(prediction_lcmv_data))

# Get predictions and names in order
predictions.append([prediction_elor])
predictions.append([prediction_mne])
predictions.append([prediction_lcmv])

model_names_tmp.append('eLORETA')
model_names_tmp.append('MNE')
model_names_tmp.append('LCMV')

# Plot True Source
brain = sim.source_data[idx].plot(**plot_params)
brain.add_text(0.1, 0.9, f'Ground Truth {sim.simulation_info.number_of_sources.values[0]} sources', 'title')
# Plot True EEG
evoked = sim.eeg_data[idx].average()
# evoked.plot()
evoked.plot_topomap(title='Ground Truth')
# evoked = util.get_eeg_from_source(sim.source_data[idx], fwd, info, tmin=0.)
# evoked.plot_topomap(title='Ground Truth Noiseless')

model_selection = model_names_tmp#['LCMV',]
# Plot predicted sources
for model_name, prediction in zip(model_names_tmp, predictions):
    
    if not any([model_name.lower() in model_select.lower() for model_select in model_selection]):
        continue
    error = util.batch_nmse(sim.source_data[idx].data, prediction[idx].data)
    r = util.batch_corr(sim.source_data[idx].data, prediction[idx].data)
    
    brain = prediction[idx].plot(**plot_params)

    title = f'{model_name}, error: {error:.4}, r: {r}'
    print(title)
    brain.add_text(0.1, 0.9, title, 'title')
    # Plot predicted EEG
    # evoked_esi = util.get_eeg_from_source(prediction[idx], fwd, info, tmin=0.)
    # evoked_esi.plot_topomap(title=model_name)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Simulating data based on sparse patches.


100%|██████████| 2/2 [00:00<?, ?it/s]


source data shape:  (1284, 100) (1284, 100)


100%|██████████| 2/2 [00:00<00:00, 153.91it/s]


interpolating for convdip...


...

  0%|          | 0/2 [00:00<?, ?it/s]

c:\Users\Lukas\Documents\projects\esinet\evaluate\..\esinet\util\util.py:388: RuntimeWarning: An average reference projection was already added. The data has been left untouched.
  epochs.set_eeg_reference(projection=True, verbose=verbose)#.apply_baseline(baseline=baseline)
c:\Users\Lukas\Documents\projects\esinet\evaluate\..\esinet\util\util.py:420: RuntimeWarning: Too few samples (required : 310 got : 100), covariance estimate may be unreliable
  data_cov = mne.compute_raw_covariance(raw, tmin=tmin,
c:\Users\Lukas\Documents\projects\esinet\evaluate\..\esinet\util\util.py:388: RuntimeWarning: An average reference projection was already added. The data has been left untouched.
  epochs.set_eeg_reference(projection=True, verbose=verbose)#.apply_baseline(baseline=baseline)
c:\Users\Lukas\Documents\projects\esinet\evaluate\..\esinet\util\util.py:420: RuntimeWarning: Too few samples (required : 310 got : 100), covariance estimate may be unreliable
  data_cov = mne.compute_raw_covariance(ra

<RawArray | 61 x 500 (5.0 s), ~325 kB, data loaded> 4.0 info empirical
<RawArray | 61 x 500 (5.0 s), ~325 kB, data loaded> 4.0 info empirical
LSTM, error: 0.01306, r: 0.29369567803768765
Fully-Connected, error: 0.01448, r: 0.2830505208690953
ConvDip, error: 0.01621, r: 0.2275248090530335
eLORETA, error: 0.08533, r: 0.1418113207347428
MNE, error: 0.03687, r: 0.15409477732151416
LCMV, error: 0.03228, r: 0.09256392265467585
Using control points [1.07776714e-10 4.75371353e-10 1.26869133e-09]


  File "C:\Users\Lukas\Envs\esienv\lib\site-packages\mne\viz\_brain\_brain.py", line 1348, in _on_button_release
    self.picked_renderer = self.plotter.iren.FindPokedRenderer(x, y)
AttributeError: 'RenderWindowInteractor' object has no attribute 'FindPokedRenderer'


Using control points [2.39900139e-10 3.22105381e-10 6.21186676e-10]


## Create or Load Evaluation Set

In [6]:
# n_samples = 1000
# duration_of_trial = (0.01, 2)
# method = 'standard'
# settings = dict(duration_of_trial=duration_of_trial, method=method)
# sim_test = Simulation(fwd, info, verbose=False, settings=settings).simulate(n_samples=n_samples)
# if type(duration_of_trial) == tuple:
#     sim_test.save(f'simulations\\sim_test_{n_samples}_{int(duration_of_trial[0]*100)}-{int(duration_of_trial[1]*100)}points_{method}.pkl')
# else:
#     sim_test.save(f'simulations\\sim_test_{n_samples}_{int(duration_of_trial*100)}points_{method}.pkl')

# or Load
with open(f'simulations\\sim_test_1000_1-200points_standard.pkl', 'rb') as f:
    sim_test = pkl.load(f)

In [7]:
%load_ext autoreload
%autoreload 2

from esinet.evaluate import eval_mean_localization_error, eval_nmse, eval_auc, eval_mse
from esinet.util import wrap_mne_inverse
from scipy.spatial.distance import cdist
from tqdm.notebook import tqdm
from joblib import Parallel, delayed

model_names_tmp = deepcopy(model_names)
# Predict
print('predict esinets...')
predictions = [model.predict(sim_test) for model in models]

print('predict elor')
pred_elor = wrap_mne_inverse(fwd, sim_test, method='eLORETA', add_baseline=True, 
    n_baseline=400)
model_names_tmp.append('eLORETA')
predictions.append(pred_elor)

print('predict MNE')
pred_mne = wrap_mne_inverse(fwd, sim_test, method='MNE', add_baseline=True, 
    n_baseline=400)
model_names_tmp.append('MNE')
predictions.append(pred_mne)

print('predict LCMV')
pred_lcmv = wrap_mne_inverse(fwd, sim_test, method='lcmv', 
    parallel=False, add_baseline=True, n_baseline=400)
model_names_tmp.append('LCMV')
predictions.append(pred_lcmv)

pos = util.unpack_fwd(fwd)[2]
argsorted_distance_matrix = np.argsort(cdist(pos, pos), axis=-1)

metrics = dict()
true_sources = np.concatenate([src.data for src in sim_test.source_data], axis=1).T

for prediction, model_name in tqdm(zip(predictions, model_names_tmp)):
    print('\n', model_name, ':\n')
     
    predicted_sources = np.concatenate([src.data for src in prediction], axis=1).T

    print('mle calculation....')
    mean_localization_errors = [eval_mean_localization_error(true_source, predicted_source, pos, argsorted_distance_matrix=argsorted_distance_matrix) for true_source, predicted_source in tqdm(zip(true_sources, predicted_sources))]
    print(len(mean_localization_errors), len(true_sources), len(predicted_sources))
    print('auc calculation....')
    # aucs_combined = [eval_auc(true_source, predicted_source, pos, epsilon=0.25, n_redraw=5) for true_source, predicted_source in tqdm(zip(true_sources, predicted_sources))]
    aucs_combined = Parallel(n_jobs=-1, backend='loky') \
        (delayed(eval_auc)(true_source, predicted_source, pos, epsilon=0.25, n_redraw=5)
        for true_source, predicted_source in tqdm(zip(true_sources, predicted_sources)))
    print('nmse calculation....')
    nmses = [eval_nmse(true_source, predicted_source) for true_source, predicted_source in tqdm(zip(true_sources, predicted_sources))]
    print('mse calculation....')
    mses = [eval_mse(true_source, predicted_source) for true_source, predicted_source in tqdm(zip(true_sources, predicted_sources))]

    aucs_far = [auc[1] for auc in np.array(aucs_combined)]
    aucs_close = [auc[0] for auc in np.array(aucs_combined)]
    aucs_combined = [np.nanmean([auc[0], auc[1]]) for auc in np.array(aucs_combined)]

    metric = pd.DataFrame(dict(
        mean_localization_errors=mean_localization_errors,
        aucs_combined=aucs_combined,
        aucs_far=aucs_far,
        aucs_close=aucs_close,
        nmses=nmses,
        mses=mses
        )
    )
    metric.name = model_name
    metrics[model_name] = metric
    

with open(f'results\\metrics_{len(true_sources)}_1-200points_standard.pkl', 'wb') as f:
    pkl.dump([metrics, sim_test.simulation_info], f)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
predict esinets...
interpolating for convdip...


1000it [01:07, 14.75it/s]


predict elor


  0%|          | 0/1000 [00:00<?, ?it/s]

predict MNE


...

0it [00:00, ?it/s]

c:\Users\Lukas\Documents\projects\esinet\evaluate\..\esinet\evaluate\evaluate.py:234: RuntimeWarning: invalid value encountered in true_divide
  y_est_normed = y_est / np.max(np.abs(y_est))


mse calculation....


0it [00:00, ?it/s]

C:\Users\Lukas\AppData\Local\Temp/ipykernel_16624/2414610761.py:59: RuntimeWarning: Mean of empty slice
  aucs_combined = [np.nanmean([auc[0], auc[1]]) for auc in np.array(aucs_combined)]


## Load Metrics

In [5]:
with open(f'results\\metrics_101947_1-200points_standard.pkl', 'rb') as f:
    [metrics, simulation_info] = pkl.load(f)

## Prepare Metrics

In [8]:
durs_in_samples = [round(dur*100) for dur in sim_test.simulation_info.duration_of_trials.values]

idx = 0
indices = []
for dur in durs_in_samples:
    idc = [idx, dur+idx]
    indices.append(idc)
    idx += dur
metrics_short = dict()


for method in metrics.keys():
    metrics_short[method] = pd.DataFrame(columns=metrics[method].columns)
    for id, idc in enumerate(indices):
        sample_summary = metrics[method].iloc[idc[0]:idc[1]].apply(np.nanmean, axis=0)
        sample_summary.sample_id = id
        metrics_short[method] = metrics_short[method].append(sample_summary, ignore_index=True)
    metrics_short[method].method = method
    # metrics_short[method].iloc[:, 0:6].values = np.real(metrics_short[method].iloc[:, 0:6])

dfs = [df[1] for df in list(metrics_short.items())]

for i, (key, df) in enumerate(metrics_short.items()):
    dfs[i]['method'] = [key]*df.shape[0]
    dfs[i]['sample_id'] = np.arange(df.shape[0])
df_aio = pd.concat(dfs)
df_aio.head()

C:\Users\Lukas\Envs\esienv\lib\site-packages\pandas\core\apply.py:821: RuntimeWarning: Mean of empty slice
  results[i] = self.f(v)


,mean_localization_errors,aucs_combined,aucs_far,aucs_close,nmses,mses,method,sample_id
0,23.219618,0.868293,0.942479,0.794107,0.012646,4.666444e-19,LSTM,0
1,11.719351,0.822743,0.875600,0.769886,0.008011,1.222845e-19,LSTM,1
2,14.562478,0.843674,0.921228,0.766119,0.013500,2.504530e-19,LSTM,2
3,19.684041,0.887201,0.966092,0.808311,0.012914,1.005835e-19,LSTM,3
4,19.647893,0.828699,0.875775,0.781622,0.010828,2.464672e-19,LSTM,4


# Plot Evaluation Metrics

## Boxplot Overview 

In [9]:
import seaborn as sns; sns.set(style='whitegrid', font_scale=1.2, font='helvetica')
%matplotlib qt

cols = df_aio.iloc[:, 0:6].columns
for y in cols:
    plt.figure(figsize=(11, 7))
    sns.boxplot(data=df_aio, x='method', y=y)
    plt.title(y.replace('_', ' ').title())
        

## Quadratic Scatter

In [71]:
import seaborn as sns; sns.set(style='whitegrid', font_scale=1.2, font='helvetica')
%matplotlib qt
methods_of_interest = ['LSTM Standard', 'Dense Standard']
# df_select = df_aio[df_aio['method'].str.contains('|'.join(methods_of_interest))]

cols = df_aio.iloc[:, 0:6].columns
for method_name in cols:

    vals_A = df_aio[df_aio['method'].str.contains(methods_of_interest[0])][method_name].values
    vals_B = df_aio[df_aio['method'].str.contains(methods_of_interest[1])][method_name].values
    d = {methods_of_interest[0]: vals_A, methods_of_interest[1]: vals_B,} 
    df_tmp = pd.DataFrame(d)
    plt.figure(figsize=(10, 10))
    ax = sns.scatterplot(data=df_tmp, x=methods_of_interest[0], y=methods_of_interest[1])


    xlim, ylim = (plt.xlim(), plt.ylim())
    plt.plot(xlim, ylim, '--k')
    xlim = (xlim[0]*0.95, xlim[1]*1.05)
    ylim = (ylim[0]*0.95, ylim[1]*1.05)
    plt.ylim(ylim)
    plt.xlim(xlim)
    
    # Title
    prop_higher = np.sum(vals_B > vals_A) / len(vals_A)
    cohens_d = (np.nanmean(vals_A) - np.nanmean(vals_B)) / np.mean([np.nanstd(vals_A), np.nanstd(vals_B)])
    median_diff = np.abs(np.nanmedian(vals_A-vals_B))
    method_name_title = method_name.replace('_', ' ').title()
    title = f'{method_name_title} ({methods_of_interest[1]} higher in {100*prop_higher:.1f} %)\nmedian_difference: {median_diff}\ncohens d: {abs(cohens_d):.2f}'
    plt.title(title)

    plt.gca().set_aspect('equal', adjustable='box')
    del d, df_tmp

## Dependence on anything

In [10]:
import pandas as pd
%matplotlib qt
sns.set(font_scale=1.2, font='helvetica')
# pd.DataFrame( metrics , index=model_names)
target_column = 'duration_of_trials'
binning = True
n_bins = 25


df = sim_test.simulation_info
if binning:
    minimum = np.min([np.min(arr) for arr in df[target_column].values])
    maximum = np.max([np.max(arr) for arr in df[target_column].values])
    
    bins = np.linspace(minimum, maximum*1.01, num=n_bins)
    bin_labels = [str(round(bins[i], 1)) + ' - ' + str(round(bins[i+1], 1)) for i in range(len(bins)-1)]
    new_target_column = 'bins ' + target_column
    df[new_target_column] = np.digitize(df[target_column].values, bins=bins)
    target_column = new_target_column
    
else:
    bins = list(set(df[target_column].values))
    bins[-1] *= 1.01
    bin_labels = [str(bins[i]) for i in range(len(bins))]


for i, model_name in enumerate(list(set(df_aio.method.values))):
    cols = df_aio[df_aio.method==model_name].iloc[:, 0:6].columns
    values = df_aio[df_aio.method==model_name].iloc[:, 0:6].values

    for metric_name, metric in zip(cols, values.T):
        col_name = model_name.replace(' ', '_') + '_' + metric_name.replace(' ', '_')
        df[col_name] = metric

dep_var_regex = target_column
dep_var_label = target_column.replace('_', ' ').title()
metric_names_nice = [col.replace('_', ' ').title() for col in df_aio.iloc[:, :6].columns]
for metric_name, metric_name_nice in zip(df_aio.iloc[:, :6].columns,  metric_names_nice):
    df_temp = pd.concat((df.filter(regex=dep_var_regex), df.filter(regex='_'+metric_name)), axis=1).melt(dep_var_regex, var_name='cols', value_name='vals')
    g = sns.catplot(x=dep_var_regex, y='vals', hue='cols', capsize=.2, kind='point', data=df_temp)
    g.set(xticklabels=bin_labels, ylabel=metric_name_nice, xlabel=dep_var_label)
    g._legend.remove()
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.tight_layout()

## Dependence on Duration

In [11]:
from tqdm.notebook import tqdm
n_simulations = 1000
settings = dict(duration_of_trial=1.,)
sim = Simulation(fwd, info, settings=settings).simulate(n_simulations)

times = sim.source_data[0].times

corrs = dict()
nmses = dict()
points = dict()
for i, time in tqdm(enumerate(times[:-1][::5])):
    for model, model_name in zip(models[:2], model_names[:2]):
        if not model_name in corrs.keys():
            corrs[model_name] = []    
            nmses[model_name] = []    
            points[model_name] = []
        r = []
        nmse = []
        new_sim = deepcopy(sim).crop(tmin=time)
        y_pred = model.predict(new_sim)
        
        for idx in range(n_simulations):
            r.append( util.batch_corr(sim.source_data[idx].data[:, -1][:, np.newaxis], y_pred[idx].data[:, -1][:, np.newaxis]) )
            nmse.append( util.batch_nmse(sim.source_data[idx].data[:, -1][:, np.newaxis], y_pred[idx].data[:, -1][:, np.newaxis]) )
        
        corrs[model_name].append(np.nanmean(r))
        nmses[model_name].append(np.nanmean(nmse))
        points[model_name].append(new_sim.source_data[idx].shape[1])
    


Simulating data based on sparse patches.


100%|██████████| 1000/1000 [00:00<00:00, 10333.88it/s]


source data shape:  (1284, 100) (1284, 100)


100%|██████████| 1000/1000 [00:07<00:00, 142.09it/s]


0it [00:00, ?it/s]

sup
sup
sup
sup
sup
sup
sup
sup
sup
sup
sup
sup
sup
sup
sup
sup
sup
sup
sup
sup
sup
sup
sup
sup
sup
sup
sup
sup
sup
sup
sup
sup
sup
sup
sup
sup
sup
sup
sup
sup


In [19]:
%matplotlib qt
plt.figure(figsize=(8,6))

plt.subplot(211)
for model_name in model_names[:2]:
    plt.plot(points[model_name], corrs[model_name], label=model_name)

plt.xlabel('Number of available data points')
plt.title('Correlation ground truth vs prediction')
plt.legend()

plt.subplot(212)
for model_name in model_names[:2]:
    plt.plot(points[model_name], nmses[model_name], label=model_name)
plt.xlabel('Number of available data points')
plt.title('NMSE ground truth vs prediction')
plt.tight_layout()
plt.legend()

## Save all figures

In [ ]:
from matplotlib.backends.backend_pdf import PdfPages

def multipage(filename, figs=None, dpi=300, png=False):
    ''' Saves all open (or list of) figures to filename.pdf with dpi''' 
    pp = PdfPages(filename)
    path = os.path.dirname(filename)
    fn = os.path.basename(filename)[:-4]

    if figs is None:
        figs = [plt.figure(n) for n in plt.get_fignums()]
    for i, fig in enumerate(figs):
        print(f'saving fig {fig}\n')
        fig.savefig(pp, format='pdf', dpi=dpi)
        if png:
            fig.savefig(f'{path}\\{i}_{fn}.png', dpi=600)
    pp.close()

multipage(f'figures\\figs_5000_10samples\\figs_5000_10samples.pdf', png=True)

## Speed test

In [ ]:
import time

def new_sim_params(sr=100, packages_per_second=20):
    package_size = int( round( sr / packages_per_second  ) )
    package_interval = package_size/sr

    n_chan = len(sim_test.eeg_data.ch_names)
    data_package = np.random.randn(n_chan, package_size)

    sim_data_package = Simulation(fwd, info, settings=dict(duration_of_trial=0.01*package_size)).simulate(1)
    print(f'performing predictions {packages_per_second} times per second')

    return sim_data_package, package_interval

packages_per_second = 50
sim_data_package, package_interval = new_sim_params(packages_per_second=packages_per_second)

while True:
    start = time.time()
    # stc = net_dense.predict(sim_data_package)
    stc = models[0].predict(sim_data_package)

    stop = time.time()
    diff = stop-start
    if stop-start > package_interval:
        print(f"took longer than expected: {diff} (instead of {package_interval})")
        print(f'decreasing package interval by one')
        packages_per_second -= 1
        sim_data_package, package_interval = new_sim_params(packages_per_second=packages_per_second)
        print(f'packages_per_second={packages_per_second}\n')
        continue
    time.sleep(package_interval-diff)

